In [1]:
import os 
import pandas as pd
from params import *

In [2]:
rental_prices = pd.read_csv(f'{DATA_DIR}/rental-prices/rental-prices.csv')
wages = pd.read_csv(f'{DATA_DIR}/ashe/weekly-earnings.csv')

In [3]:
rental_prices = rental_prices[(rental_prices.date == max(rental_prices.date)) 
                              &(rental_prices.variable_code == 'Lower quartile')
                              &(rental_prices.property_name == 'bed2')]
rental_prices = rental_prices[['geography_code', 'value']].set_index('geography_code')
rental_prices.rename(columns={'value': 'lower_quartile_rent'}, inplace=True)
rental_prices

,lower_quartile_rent
geography_code,
E92000001,625
E12000001,450
E06000047,400
E06000005,438
E06000001,400
...,...
E10000027,650
E07000187,700
E07000188,650


In [4]:
wages = wages[
    (wages.date == max(wages.date)) 
    &(wages.variable_name == 'lq_monthly_wage')]
wages = wages[['geography_code', 'geography_name', 'value']].set_index('geography_code')
wages.rename(columns={'value': 'lq_monthly_wage'}, inplace=True)
wages

,geography_name,lq_monthly_wage
geography_code,,
E12000001,North East,1539.0
E06000005,Darlington UA,1564.0
E06000001,Hartlepool UA,1344.0
E06000002,Middlesbrough UA,1387.0
E06000003,Redcar and Cleveland UA,1278.0
...,...,...
E08000032,Bradford,1472.0
E08000033,Calderdale,1540.0
E08000034,Kirklees,1513.0


In [5]:
merged = wages.merge(rental_prices, left_index=True, right_index=True)
print(merged.columns.to_list())
merged['lq_rent_to_lq_earnings'] = merged['lower_quartile_rent'] / merged['lq_monthly_wage']
merged

['geography_name', 'lq_monthly_wage', 'lower_quartile_rent']


,geography_name,lq_monthly_wage,lower_quartile_rent,lq_rent_to_lq_earnings
geography_code,,,,
E12000001,North East,1539.0,450,0.292398
E06000005,Darlington UA,1564.0,438,0.280051
E06000001,Hartlepool UA,1344.0,400,0.297619
E06000002,Middlesbrough UA,1387.0,450,0.324441
E06000003,Redcar and Cleveland UA,1278.0,448,0.350548
...,...,...,...,...
E08000032,Bradford,1472.0,475,0.322690
E08000033,Calderdale,1540.0,475,0.308442
E08000034,Kirklees,1513.0,475,0.313946


In [6]:
merged = merged.melt(value_vars='lq_rent_to_lq_earnings', 
                    id_vars='geography_name', 
                    ignore_index=False, 
                    var_name='variable_name'
                    )
merged = merged.dropna()
# merged.to_csv(f'{DATA_DIR}/ashe/rent-earning-ratio.csv')

In [7]:
PARQUET_FILE = '../../../data-mart/ashe/rent-earning-ratio.parquet'
os.makedirs(os.path.dirname(PARQUET_FILE), exist_ok=True)
merged.to_parquet(PARQUET_FILE)